In [23]:
# install requirements (replace this later with requirements.txt)

## run this cell to install prosodic, then restart runtime as requested

%pip install -U prosodic
#%pip install openai


/bin/bash: apt-get: command not found
Note: you may need to restart the kernel to use updated packages.


In [8]:
# set api keys
import os
os.environ["OPENAI_KEY"] = "ADD HERE"


In [17]:
# prompt dictionary generator

all_prompt_dicts = []

# syntax is "write me a line [meter_prompts] including the word [seed_word][variations] "
# can add random words

meter_prompts = ["in iambic pentameter", "of blank verse", "in meter"]
variations = ["","with rhythmic variation", "with a trochaic inversion", "with a trochaic inversion after a caesura", "with an inverted foot", "with a trochaic inversion at the end of the line"]

word_list = ["sorrow", "lament", "truth", "lie", "love", "hate", "beginning", "ultimate", 'reason', 'deceit']



#prompt generator

for meter in meter_prompts:
  for variation in variations:
    for seed_word in word_list:
      # Change this line to define a dictionary
      possible_prompts = {"prompt": f"write a line {meter} including the word {seed_word} {variation}"}

      for key, value in possible_prompts.items():
        all_prompt_dicts.append({'meter': meter,  # Changed meter_prompt to meter
                                'variation': variation,
                                'seed_word': seed_word,
                                'prompt': value
                                                  })

In [18]:
#openai api
#%pip install openai

import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_KEY"),  # replace with your API key
)

# Assuming you have set your OpenAI API key

for prompt_dict in all_prompt_dicts:
    try:
        # Create completion using OpenAI API
        completion = client.chat.completions.create(
            model="o1-mini-2024-09-12",
            messages=[
               # {"role": "system", "content": "Generate exactly one line of poetry based on the prompt."},
                {
                    "role": "user", 
                    "content": "Reply with only the line of poetry requested."+ prompt_dict['prompt']}
            ]
        )
        
        # Extract the generated line and add it to the dictionary
        print(completion.choices[0].message.content.strip())
        prompt_dict['response'] = completion.choices[0].message.content.strip()
        all_prompt_dicts.append(prompt_dict)
        
    except Exception as e:
        print(f"Error processing prompt: {prompt_dict['prompt']}")
        print(f"Error message: {str(e)}")
        prompt_dict['response'] = "Error generating response"



Within her heart, a veil of deep sorrow
In shadows deep, my heart begins to lament.
The morning sun reveals the honest truth.
She whispered secrets that could not lie.
In every breath, the essence of our love.
The shadows cast by hate obscure our way
In every dawn, we find a new beginning.
Our journey leads to the ultimate truth
Within her eyes, I find the light of reason.
Beneath the moon, the shadows weave deceit.
The sorrow in her eyes betrayed her heart.
The autumn leaves descend in mournful lament
Beneath the moon, the truth unveils its grace.
Beneath the silver moon, our secrets lie.
Embracing love beneath the silver moon.
In shadows deep, my heart conceals its hate
Beginning bright, the stars ignite the night.
In shadows deep, we chase our ultimate.
Though reason fades, the heart still finds its way
Amid the shadows, deceit weaves its dark spell.
Sorrow grips the heart with silent pain
Lament fills the valleys deep with silent tears.
Truth shines through darkest night with stead

KeyboardInterrupt: 

In [ ]:
## cell for other apis / models

In [20]:
#if desired, convert dict to df or csv

import pandas as pd

# Convert the list of dictionaries to a DataFrame
df_prompts = pd.DataFrame(all_prompt_dicts)

# Display the DataFrame
#df_prompts
df_prompts.drop_duplicates()
df_prompts.to_csv('prompt_responses.csv', index=False)

In [ ]:
df_prompts

In [30]:
pd.read_csv('/Users/bsg25/metricalgpt/prompt_responses.csv',encoding='latin-1')

,in meter,with a trochaic inversion,beginning,write a line in meter including the word beginning with a trochaic inversion,Alone the beginning casts its silent spell.
0,in meter,with a trochaic inversion,lament,write a line in meter including the word lamen...,Alone the stars mournful lament
1,in iambic pentameter,with rhythmic variation,sorrow,write a line in iambic pentameter including th...,"Amid the night, my heart knows deepest sorrow."
2,in meter,with rhythmic variation,truth,write a line in meter including the word truth...,"Amid the shadows, truth unveils its light."
3,of blank verse,with a trochaic inversion at the end of the line,truth,write a line of blank verse including the word...,"Amid the storm, the steadfast heart reveals truth"
4,in iambic pentameter,with an inverted foot,ultimate,write a line in iambic pentameter including th...,"Amid the ultimate dusk, the stars align"
...,...,...,...,...,...
177,in meter,with a trochaic inversion,deceit,write a line in meter including the word decei...,Within deceit the silent shadows fall
178,of blank verse,with rhythmic variation,hate,write a line of blank verse including the word...,"Within his soul, the heavy chains of hate."
179,in iambic pentameter,with a trochaic inversion at the end of the line,hate,write a line in iambic pentameter including th...,"Within my heart, the seeds of hate do grow."
180,in iambic pentameter,NaN,reason,write a line in iambic pentameter including th...,"Within our hearts, we find the strength of reason"


In [33]:
import pandas as pd
import prosodic

# Read the "response" column of "prompt_responses" and add the output to the CSV

prosodic.logger.setLevel('ERROR')


# Set meter. Defaults here force iambic pentameter and prioritize the rule against lexically "strong" peaks (w_peak)"
# try other configurations to align with your sense of the meter!

constraints = {
    'w_peak': 2.0,
    'w_stress': 1.0,
    's_unstress': 1.0,
    'unres_across': 1.0,
    'unres_within': 0.0,
    'pentameter': 10.0,
    'iambic': 10.0,
}
meter = prosodic.Meter(
    constraints=constraints,
    resolve_optionality=True,
    max_s=1,
    max_w=2,
)

# Create an empty DataFrame with the desired column headers
column_headers = ["line_txt", "txt", "meter", "stress", "score", "num_viols", "ambig"]
df = pd.DataFrame(columns=column_headers)

# Loop through the "response" column of prompt_responses and add data to DataFrame
for response in df_prompts['response']:
    response = response.strip()  # Remove leading/trailing whitespace

    response = prosodic.Text(response)  # Process with prosodic.Text()
    parses = response.parse()
    parse_dict = parses[0].best_parse.attrs

    # Extract data from the first parse (modify if you need data from other parses)
    new_dict = {key: parse_dict[key] for key in column_headers if key in parse_dict}

    # Append data to DataFrame
    df = pd.concat([df, pd.DataFrame([new_dict])], ignore_index=True)

# Merge the new DataFrame with the original prompt_responses DataFrame
merged_df = df_prompts.merge(df, left_on='response', right_on='line_txt', how='left')
# Display the merged DataFrame

merged_df

# Save the merged DataFrame to a CSV file [replace path]
merged_df.to_csv('/Users/bsg25/Library/CloudStorage/OneDrive-YaleUniversity/TrochAIc/AI Verse/merged_prompt_responses.csv', index=False)  # Replace with your output file path

[0.29s] Parsing lineparts [6x]: 100%|██████████| 2/2 [00:00<00:00,  9.02it/s]s]



































































































































































































































































                                                                             


































































[0.04s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                     

[0.04s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]

[0.04s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]


[0.04s] Parsing lineparts [6x]:   0%|          | 0/2 [00:00<?, ?it/s]



[0.03s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]



























































































In [35]:
merged_df = merged_df.drop_duplicates()
merged_df.to_csv('/Users/bsg25/metricalgpt/merged_to_analyze.csv')

In [ ]:
#see available openAI models

import os
import openai

client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_KEY")
)
models = client.models.list()
for model in models:
    print(model.id)

gpt-4o-audio-preview-2024-10-01
gpt-4o-mini-audio-preview
gpt-4o-realtime-preview-2024-10-01
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
gpt-4o-mini
gpt-4-1106-preview
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-instruct
babbage-002
o1-mini
o1-mini-2024-09-12
whisper-1
dall-e-3
gpt-4o-2024-08-06
gpt-4o
gpt-3.5-turbo-16k
omni-moderation-latest
o1-preview-2024-09-12
omni-moderation-2024-09-26
tts-1-hd-1106
o1-preview
gpt-4
gpt-4-0613
chatgpt-4o-latest
gpt-4-0125-preview
tts-1-hd
davinci-002
gpt-4o-mini-2024-07-18
text-embedding-ada-002
gpt-3.5-turbo-1106
gpt-4-turbo-2024-04-09
gpt-4-turbo
tts-1
tts-1-1106
gpt-3.5-turbo-instruct-0914
gpt-4-turbo-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-audio-preview
gpt-4o-2024-05-13
text-embedding-3-small
gpt-4o-2024-11-20
text-embedding-3-large
gpt-4o-audio-preview-2024-12-17
gpt-4o-realtime-preview


The cells below are for a new run of prompts and parses!

In [41]:
# prompt dictionary generator for big chatgpt-4o-latest run

all_prompt_dicts = []

# syntax is "write me a line [meter_prompts] including the word [seed_word][variations] "
# can add random words

meter_prompts = ["in iambic pentameter", "of blank verse", "in meter"]
variations = ["","with rhythmic variation", "with a trochaic inversion", "with a trochaic inversion after a caesura", "with an inverted metrical foot"]

word_list = ["bear", "whale", "sloth", "cat", "dog", "giraffe", "gazelle", "baboon", "raccoon", "macaw", "alpaca", "hyena", "chinchilla", "coyote", "gorilla", "kangaroo", "platypus", "porcupine", "pangolin", "ocelot", "lemur", "lemming", "otter", "parrot", "squirrel"]



#prompt generator

for meter in meter_prompts:
  for variation in variations:
    for seed_word in word_list:
      # Change this line to define a dictionary
      possible_prompts = {"prompt": f"write a line {meter} including the word {seed_word} {variation}"}

      for key, value in possible_prompts.items():
        all_prompt_dicts.append({'meter': meter,  # Changed meter_prompt to meter
                                'variation': variation,
                                'seed_word': seed_word,
                                'prompt': value
                                                  })

In [42]:
#openai api
#%pip install openai

import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_KEY"),  # replace with your API key
)

# Assuming you have set your OpenAI API key

for prompt_dict in all_prompt_dicts:
    try:
        # Create completion using OpenAI API
        completion = client.chat.completions.create(
            model="chatgpt-4o-latest",
            messages=[
               # {"role": "system", "content": "Generate exactly one line of poetry based on the prompt."},
                {
                    "role": "user", 
                    "content": "Reply with only the line of poetry requested."+ prompt_dict['prompt']}
            ]
        )
        
        # Extract the generated line and add it to the dictionary
        print(completion.choices[0].message.content.strip())
        prompt_dict['response'] = completion.choices[0].message.content.strip()
        all_prompt_dicts.append(prompt_dict)
        
    except Exception as e:
        print(f"Error processing prompt: {prompt_dict['prompt']}")
        print(f"Error message: {str(e)}")
        prompt_dict['response'] = "Error generating response"



The forest whispers soft, where dwells the bear.
The mighty whale glides through the ocean deep.
The gentle sloth descends the branch with care.
The slender cat now prowls beneath the moon.
The loyal dog does wait beneath the moon.
Beneath the sun, the graceful giraffe strides.
The graceful leap, a gazelle in the night.
The cunning baboon danced beneath the moon.
Beneath the moon, I spied a sly raccoon.
The vibrant macaw sings beneath the skies.
The soft alpaca roams the mountain high.
The shadowed hyena prowls beneath the moon.
A quiet chinchilla dreams beneath the moon.
The mournful cries of midnight’s lone coyote.
The fearful gorilla hides deep in the shade.
Upon the plains, I spied a kangaroo.
A curious platypus glides through the stream.
Beware the quiet steps of porcupine.
Beneath the moon, the pangolin doth crawl.
Beneath the moon, the ocelot does prowl.
The lemur leaps through trees with grace unseen.
The lemming leaps into the restless tide.
The playful otter glides through st

KeyboardInterrupt: 

In [45]:
#convert to df and drop duplicates

df_4o_latest = pd.DataFrame(all_prompt_dicts)
df_4o_latest.drop_duplicates()




,meter,variation,seed_word,prompt,response
0,in iambic pentameter,,bear,write a line in iambic pentameter including th...,The weary bear descends the mountain's height.
1,in iambic pentameter,,whale,write a line in iambic pentameter including th...,"Beneath the waves, the mighty whale does glide."
2,in iambic pentameter,,sloth,write a line in iambic pentameter including th...,The sloth ascends the tree with gentle grace.
3,in iambic pentameter,,cat,write a line in iambic pentameter including th...,The moonlight danced upon the silent cat.
4,in iambic pentameter,,dog,write a line in iambic pentameter including th...,The loyal dog awaits beside the fire.
...,...,...,...,...,...
370,in meter,with an inverted metrical foot,lemur,write a line in meter including the word lemur...,Amid the shadows leaps the spry lemur.
371,in meter,with an inverted metrical foot,lemming,write a line in meter including the word lemmi...,Headlong the lemming dives from cliffs to sea.
372,in meter,with an inverted metrical foot,otter,write a line in meter including the word otter...,"Upon the stream, the otter sleek does glide."
373,in meter,with an inverted metrical foot,parrot,write a line in meter including the word parro...,"Upon the perch, the parrot shrieks at dawn."


In [50]:
##cleaning cell

# Fill missing 'variation' values with "none"
df_4o_latest['variation'].fillna('none', inplace=True)
df_4o_latest = df_4o_latest.drop_duplicates()



In [52]:
import pandas as pd
import prosodic

# Read the "response" column of "prompt_responses" and add the output to the CSV

prosodic.logger.setLevel('ERROR')


# Set meter. Defaults here force iambic pentameter and prioritize the rule against lexically "strong" peaks (w_peak)"
# try other configurations to align with your sense of the meter!

constraints = {
    'w_peak': 2.0,
    'w_stress': 1.0,
    's_unstress': 1.0,
    'unres_across': 1.0,
    'unres_within': 0.0,
    'pentameter': 10.0,
    'iambic': 10.0,
}
meter = prosodic.Meter(
    constraints=constraints,
    resolve_optionality=True,
    max_s=1,
    max_w=2,
)

# Create an empty DataFrame with the desired column headers
column_headers = ["line_txt", "txt", "meter", "stress", "score", "num_viols", "ambig"]
df = pd.DataFrame(columns=column_headers)

# Loop through the "response" column of prompt_responses and add data to DataFrame
for response in df_4o_latest['response']:
    response = response.strip()  # Remove leading/trailing whitespace

    response = prosodic.Text(response)  # Process with prosodic.Text()
    parses = response.parse()
    parse_dict = parses[0].best_parse.attrs

    # Extract data from the first parse (modify if you need data from other parses)
    new_dict = {key: parse_dict[key] for key in column_headers if key in parse_dict}

    # Append data to DataFrame
    df = pd.concat([df, pd.DataFrame([new_dict])], ignore_index=True)

# Merge the new DataFrame with the original prompt_responses DataFrame
df_4o_latest_merged = df_4o_latest.merge(df, left_on='response', right_on='line_txt', how='left')
# Display the merged DataFrame

# Save the merged DataFrame to a CSV file [replace path]
#merged_df.to_csv('/FILEPATH, index=False)  # Replace with your output file path

[3632.81s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]

































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [54]:
# Add "none" if variation is blank, then delete rows with "missing value"
df_4o_latest_merged['variation'] = df_4o_latest_merged['variation'].replace('', 'none')
df_4o_latest_merged = df_4o_latest_merged.dropna()

In [ ]:
df_4o_latest_merged.to_csv('/Users/bsg25/metricalgpt/merged_to_analyze_4o_latest.csv')